In [ ]:
# ohlcs.py
"""Program that generates ohlcs
Date: 23-June-2019
Ver: 1.0
Time taken: 
 - all from IBKR: 8 mins
 - from NSE Web:
"""
from z_helper import *  # needed for util

def ohlcs(ib, id_sym, fspath, logpath):
    '''Get ohlcs with stDev (8 mins)
    Args:
        (ib) as connection object
        (id_sym) as {undId: 'symbol'} dictionary
        (logpath) as string for path for lots
        (fspath) as string for path of pickles
    Returns:
        ohlcs dataframe with stDev'''

    with open(logpath+'ohlc.log', 'w'):
        pass # clear the run log

    util.logToFile(logpath+'ohlc.log')

    ohlcs = []
    with tqdm(total= len(id_sym), file=sys.stdout, unit= 'symbol') as tqh:
        for k, v in id_sym.items():
            tqh.set_description(f"Getting OHLC hist frm IBKR for {v.ljust(9)}")
            ohlcs.append(catch(lambda:do_hist(ib, k, fspath)))
            tqh.update(1)

    # Remove nan from ohlcs list
    li = [o for o in ohlcs if str(o) != 'nan']
    
    df_ohlcs = pd.concat(li).reset_index(drop=True)
    
    return df_ohlcs

In [ ]:
# test code for NSE ohlcs

from z_helper import *
util.startLoop()

# from json (for logpath)
a = assign_var('nse')
for v in a:
    exec(v)

from chains_nse import *
from ohlcs import *

if __name__ == '__main__':
    
    #... check for chains_nse.pkl
    if path.isfile(fspath+'chains_nse.pkl'):
        df_chains = pd.read_pickle(fspath+'chains_nse.pkl')
    else:
        df_chains = get_chains(nseweb=False)  # get nse purely from a list. Not from nse website

#     df_chains = df_chains[df_chains.symbol.isin(['BANKNIFTY', 'PNB'])]  # DATA LIMITER!!!

    id_sym = df_chains.set_index('undId').symbol.to_dict()
    
    with get_connected('nse', 'live') as ib:
        df_ohlcs = ohlcs(ib, id_sym, fspath, logpath)

df_ohlcs.to_pickle(fspath+'ohlcs.pkl')

In [1]:
# test code for SNP ohlcs

from z_helper import *
util.startLoop()

# from json (for logpath)
a = assign_var('snp')
for v in a:
    exec(v)

from chains_snp import *
from ohlcs import *

if __name__ == '__main__':
    
    #... check for chains_snp.pkl
    if path.isfile(fspath+'chains_snp.pkl'):
        df_chains = pd.read_pickle(fspath+'chains_snp.pkl')
    else:
        with get_connected('snp', 'live') as ib:
            df_chains = get_chains(ib)  # get snp purely from a list. Not from snp website

#     df_chains = df_chains[df_chains.symbol.isin(['BANKNIFTY', 'PNB'])]  # DATA LIMITER!!!

    id_sym = df_chains.set_index('undId').symbol.to_dict()
    
    with get_connected('snp', 'live') as ib:
        df_ohlcs = ohlcs(ib, id_sym, fspath, logpath)

df_ohlcs.to_pickle(fspath+'ohlcs.pkl')

Getting OHLC hist frm IBKR for XOM      : 100%|████████████████████████████████████| 91/91 [03:13<00:00,  1.11symbol/s]


In [ ]:
# Capture the ohlc pickles from pickled files (manually)
ohlc_pkls = [f for f in listdir(fspath) if (f[-8:] == 'ohlc.pkl')]
df = pd.concat([pd.read_pickle(fspath+o) for o in ohlc_pkls]).reset_index(drop=True)
df.to_pickle(fspath+'ohlcs.pkl')